# Monad(not Monoid)
アプリカティブ値の自然な拡張。アプリカティブ値と、普通の値を取るけどアプリカティブな値を返す関数があった時、前者を後者に食わせることができる。
```haskell
instance Monad Maybe where
    return x = Just x
    Nothing >>= f = Nothing --関数適用(Nothingの場合)
    Just x >>= f = f x --関数適用
    fail _ = Nothing --失敗した時の関数
```

In [1]:
Just 9 >>= \x -> return (x*10)
Nothing >>= \x -> return (x*10)

Line 2: Evaluate
Found:
Nothing >>= \ x -> return (x * 10)
Why not:
Nothing

Just 90

Nothing

棒の左右にとまった鳥の数の差が3以内であれば、ピエールはバランスを取れるものとしましょう。鳥たちがバランス棒の左右の端に飛んできたり飛び去ったりするようすをシミュレートし、一定数の鳥たちが来たり去ったりした後も、ピエールが綱の上にいるかどうか判定するプログラムを書いてみましょう。

In [15]:
type Birds = Int
type Pole = (Birds,Birds)
landLeft :: Birds -> Pole -> Maybe Pole
landLeft n (left,right)
    | abs((left+n)-right)<4 = Just (left + n ,right)
    | otherwise = Nothing
landRight :: Birds -> Pole -> Maybe Pole
landRight n (left,right)
    | abs(left-(right+n))<4 = Just (left,right+n)
    | otherwise = Nothing
x -: f = f x

In [16]:
landRight 1 (0,0)   >>= landLeft 2
Nothing  >>=  landLeft 2

Line 2: Evaluate
Found:
Nothing >>= landLeft 2
Why not:
Nothing

Just (2,1)

Nothing

In [21]:
return (0, 0) >>= landRight 2 >>= landLeft 2 >>= landRight 2
return (0, 0) >>= landLeft 1 >>= landRight 4 >>= landLeft (-1) >>= landRight (-2)
return (0, 0) >>= landRight 2 >> Nothing >>= landLeft 2

Line 1: Monad law, left identity
Found:
return (0, 0) >>= landRight 2
Why not:
landRight 2 (0, 0)Line 2: Monad law, left identity
Found:
return (0, 0) >>= landLeft 1
Why not:
landLeft 1 (0, 0)Line 3: Monad law, left identity
Found:
return (0, 0) >>= landRight 2
Why not:
landRight 2 (0, 0)

Just (2,4)

Nothing

Nothing

## do記法
モナドの結果（例えばJust 3なら3、NothingならNothingを返す）を変数に束縛するには`<-`を使う

In [27]:
foo :: Maybe String
foo = do
   x<- Just 3
   y<- Just "!"
   Just (show x ++ y)
foo
bar :: Maybe String
bar= do
   x <- Just 3
   y<- Just "!"
   Nothing
   Just (show x ++ y)
bar

Just "3!"

Nothing

### do記法の中のfail
do記法の中でパターンマッチに失敗するとfailが呼ばれる。Maybeの時はNothing。

In [28]:
wopwop :: Maybe Char
wopwop = do
    (x:xs) <- Just ""
    return x
wopwop

Nothing

## リストのモナド
```
instance Monad [] where
    return x = [x]
    xs >>= f = concat (map f xs)
    fail _ = []
```

In [32]:
[3,4,5] >>= \x -> [x,-x]
[3,4,5] >>= \x -> []
[1,2] >>= \n -> ['a','b'] >>= \ch -> return (n,ch)

Line 2: Use const
Found:
\ x -> []
Why not:
const []

[3,-3,4,-4,5,-5]

[]

[(1,'a'),(1,'b'),(2,'a'),(2,'b')]

## guard
guardは引数がFalseなら失敗を投げ、Trueならダミーの値が入っている成功を作る

In [38]:
import Control.Applicative
import Control.Monad
[1..50] >>= (\x -> guard ('7'  `elem` show x) >> return x)

[7,17,27,37,47]

チェス盤の上にナイトの駒が一つだけ乗っていて、ナイトを3回動かして特定のマスまで移動させられるかを調べる

In [46]:
type KnightPos = (Int,Int)
moveKnight :: KnightPos -> [KnightPos]
moveKnight (c,r) = do
    (c', r') <- [(c+2,r-1),(c+2,r+1),(c-2,r-1),(c-2,r+1),(c+1,r-2),(c+1,r+2),(c-1,r-2),(c-1,r+2)]
    guard (c' `elem` [1..8] && r' `elem` [1..8])
    return (c', r')
in3 :: KnightPos -> [KnightPos]
in3 start = return start >>= moveKnight >>= moveKnight >>= moveKnight
canReachIn3 start end = end `elem` in3 start

Line 8: Monad law, left identity
Found:
return start >>= moveKnight
Why not:
moveKnight start

In [48]:
(6,2) `canReachIn3 ` (6,1)
(6,2) `canReachIn3 ` (7,3)

True

False

演習:経路を教えてくれるように改造する。
## モナド則
- 左恒等性:`return x >>= f = f x`
- 右恒等性:`m >>= return = m`
- 結合法則 :`(m>>=f)>>=g`と`m >>= (\x -> f x >>= g)`